In [1]:
import traja
from traja import TrajaCollection
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit

In [2]:
# create trajectories with shorter step lengths

num_short = 5
x_short_nps = []
y_short_nps = []
short_trjs = {ind: traja.generate(n=1000, random=True,step_length= 2, seed=ind) for ind in range(num_short)}
for trj in short_trjs:
    derivs = traja.get_derivatives(short_trjs[trj])
    ang = traja.calc_angle(short_trjs[trj])
    short_trjs[trj] = short_trjs[trj].join(derivs).join(ang.rename('angles'))
    short_trjs[trj].drop(['displacement_time', 'speed_times', 'acceleration_times'], inplace=True, axis=1)
    # print(short_trjs[trj][short_trjs[trj].isnull().any(axis=1)])
    
    short_trjs[trj].to_numpy
    x_short_nps.append(short_trjs[trj])
    y_short_nps.append(0)
    

x_short_data = np.dstack(x_short_nps)
print(x_short_data.shape)


(1000, 7, 5)


In [3]:
# create trajectories with longer step lengths
# data set now has shorter and longer step lengths

num_long = 5
x_long_nps = [x_short_data]
long_trjs = {ind: traja.generate(n=1000, random=True,step_length= 5, seed=ind+num_short) for ind in range(num_long)}
for trj in long_trjs:
    derivs = traja.get_derivatives(long_trjs[trj])
    ang = traja.calc_angle(long_trjs[trj])
    long_trjs[trj] = long_trjs[trj].join(derivs).join(ang.rename('angles'))
    long_trjs[trj].drop(['displacement_time', 'speed_times', 'acceleration_times'], inplace=True, axis=1)
    long_trjs[trj].to_numpy
    x_long_nps.append(long_trjs[trj])
    y_short_nps.append(1)

X = np.dstack(x_long_nps)
X = np.transpose(X)
Y = np.array(y_short_nps)
print(X.shape)
print(Y.shape)

(10, 7, 1000)
(10,)


In [4]:
# shuffle + split data set into training and testing set
from sklearn.model_selection import train_test_split
all_indices = list(range(num_short+num_long))
train_ind, test_ind = train_test_split(all_indices, test_size=0.5)
print(train_ind)
print(test_ind)

X_train = X[train_ind, :, 2:]
X_test = X[test_ind, :, 2:]
Y_train = Y[train_ind]
Y_test = Y[test_ind]




[0, 1, 7, 8, 4]
[3, 9, 2, 6, 5]


In [5]:
# Try time-series concatenation
from sktime.transformations.panel.compose import ColumnConcatenator
from sklearn.pipeline import Pipeline
from sktime.classification.interval_based import TimeSeriesForestClassifier


steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=10)),
]
clf = Pipeline(steps)

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)

output = cross_validate(clf, X[:,:,2:], Y, cv=k_fold, scoring = 'roc_auc', return_estimator =True)
print(output)

ModuleNotFoundError: No module named 'sktime'

In [57]:
ColumnConcatenator.transform(X[:,:,2:])

TypeError: transform() missing 1 required positional argument: 'X'

In [54]:
clf.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
Y_pred = clf.predict(X_test)
print(Y_pred)
print(Y_test)
print(score)

[0 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0
 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1
 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 1
 1 1 1 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 1 1 1 0 0
 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1
 1 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 0 0 1 0 1 0 1 1 1 1 1 0 1 0 1 0 0 1
 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0 0 0 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 1 0 1 0 1
 0 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1
 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1
 0 1 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1
 0 0 1 0 0 1 0 0 0 1 1 1 

In [170]:
# Use rocket Classifier
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score

rocket = RocketClassifier()
rocket.fit(X_train, Y_train)
Y_pred = rocket.predict(X_test)
accuracy_score(Y_test, Y_pred)



0.375

In [155]:
# Try Column Ensembling
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
clf = ColumnEnsembleClassifier(
    estimators=[
        ("TSF0", TimeSeriesForestClassifier(n_estimators=10), [0]),
        ("BOSSEnsemble3", BOSSEnsemble(max_ensemble_size=5), [3]),
    ]
)
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

KeyboardInterrupt: 

In [126]:
!ln -s /Users/kellyfang/opt/anaconda3/envs/env_pytorch/lib/python3.6/site-packages/sktime/distances/elastic_cython.cpython-36m-darwin.so /usr/local/opt/libomp/lib/libomp.dylib

ln: /usr/local/opt/libomp/lib/libomp.dylib: File exists


In [130]:
# try time-series clustering
import sktime.clustering as cluster
b = cluster.BaseClusterer(n_cluster=2)
b.predict(X_train)

AttributeError: module 'sktime.clustering' has no attribute 'BaseClusterer'

ERROR: Could not find a version that satisfies the requirement sktime==0.11.4 (from versions: 0.1.dev0, 0.1.0, 0.2.0, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.5.0, 0.5.1, 0.5.2, 0.5.3, 0.6.0, 0.6.1, 0.7.0, 0.8.0, 0.8.1, 0.9.0)
ERROR: No matching distribution found for sktime==0.11.4
